From https://www.kaggle.com/chauhuynh/my-first-kernel-3-699

In [11]:
import numpy as np
import pandas as pd
import datetime
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE_TSG = 10007
np.random.seed(RANDOM_STATE_TSG)

In [2]:
file_path = 'F:/github_me_repos/Kaggle_code/elo-merchant-category-recommendation/'

In [3]:
df_train = pd.read_csv('./train_first_kernel.csv')
df_test = pd.read_csv('./test_first_kernel.csv')

print('finished!')

finished!


In [4]:
df_train.shape, df_test.shape
# df_train['hist_purchase_date_max'].head(20)
# df_train.dtypes

0     1.519551e+09
1     1.517438e+09
2     1.519759e+09
3     1.519818e+09
4     1.519850e+09
5     1.501343e+09
6     1.519402e+09
7     1.513885e+09
8     1.512825e+09
9     1.519837e+09
10    1.519369e+09
11    1.514577e+09
12    1.519021e+09
13    1.508012e+09
14    1.509404e+09
15    1.519480e+09
16    1.519798e+09
17    1.517915e+09
18    1.517765e+09
19    1.519497e+09
Name: hist_purchase_date_max, dtype: float64

In [5]:
# df_train_no_outliers = df_train[df_train['outliers']==0]
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [6]:
# https://www.kaggle.com/adarshchavakula/out-of-fold-oof-model-cross-validation

# # Read Data
# print("Reading Dataset...")
# train = pd.read_csv("../input/train.csv")
# target = np.array(train["target"])
# target_log = np.log1p(target) # Log transform target as the evaluation metric uses it
# xtrain = np.array(train.iloc[:,2:])
# print("Shape of training data: {}".format(np.shape(xtrain)))

# # Define Model 
# xgb_model = XGBRegressor(max_depth=6, learning_rate=0.1, n_estimators=70,
#                          min_child_weight=100, subsample=1.0, 
#                          colsample_bytree=0.8, colsample_bylevel=0.8,
#                          random_state=42, n_jobs=4)

# # Make OOF predictions using 5 folds
# print("Cross Validating...")
# oof_preds_log = cross_val_predict(xgb_model, xtrain, target_log, cv=5, 
#                                   n_jobs=1, method="predict")
                                  
# # Calculate RMSLE (RMSE of Log(1+y))
# cv_rmsle = np.sqrt(mean_squared_error(target_log, oof_preds_log))
# print("\nOOF RMSLE Score: {:.4f}".format(cv_rmsle))

In [13]:
RANDOM_STATE_TSG = 10007
target = df_train['outliers']

features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers', 'target']]
features_old = features
features = list(df_train[features].select_dtypes(include=['number', 'bool']).columns.values)  # 去除非数值、非布尔类型数据
# categorical_feats = [c for c in features if 'feature_' in c]

categorical_feats = []
print(len(features), len(categorical_feats))

param_classifier = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333
}


param_regressor = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333
}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    
    # 训练一个分类器 决定是否为 outlier
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    auc = roc_auc_score(target.iloc[val_idx], oof[val_idx])
    print('auc is ', auc)
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))
print('final auc is ', roc_auc_score(target, oof))

# CV score: 0.04615 
# final auc is  0.8738057712001387

predictions_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
outlier_num = int(df_train['outliers'].mean()*df_test.shape[0])  # outlier 个数
print('outlier_num is ', outlier_num)
predictions_is_outlier['target'] =  predictions

outlier_id = list(predictions_is_outlier.sort_values(by='target',ascending = False).head(outlier_num)['card_id'])
print('len of outlier_id is ', len(outlier_id), 'first 20 of outlier_id is ', outlier_id[:20])

103 0
fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0445171	valid_1's binary_logloss: 0.0471901
[200]	training's binary_logloss: 0.0445063	valid_1's binary_logloss: 0.0471852
Early stopping, best iteration is:
[5]	training's binary_logloss: 0.0444367	valid_1's binary_logloss: 0.0471405
auc is  0.8715427047639518
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0445582	valid_1's binary_logloss: 0.0454151
[200]	training's binary_logloss: 0.0445547	valid_1's binary_logloss: 0.0454116
Early stopping, best iteration is:
[48]	training's binary_logloss: 0.0445184	valid_1's binary_logloss: 0.0453669
auc is  0.8782335510650503
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0446811	valid_1's binary_logloss: 0.0442317
[200]	training's binary_logloss: 0.0446659	valid_1's binary_logloss: 0.0442
Early stopping, best iterati

In [ ]:
RANDOM_STATE_TSG = 10007
target = df_train['outliers']

features = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','outliers', 'target']]
features = list(df_train[features].select_dtypes(include=['number', 'bool']).columns.values)  # 去除非数值、非布尔类型数据
# categorical_feats = [c for c in features if 'feature_' in c]

categorical_feats = []
print(len(features), len(categorical_feats))

param_classifier = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "rf",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "random_state": 2333
}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE_TSG)
# folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train['outliers'].values)):
    print("fold n°{}".format(fold_))
    
    # 训练一个分类器 决定是否为 outlier
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=df_train['outliers'].iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=df_train['outliers'].iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    auc = roc_auc_score(target.iloc[val_idx], oof[val_idx])
    print('auc is ', auc)
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))
print('final auc is ', roc_auc_score(target, oof))

# CV score: 0.04615 
# final auc is  0.8738057712001387

predictions_is_outlier = pd.DataFrame({"card_id": df_test["card_id"].values})
outlier_num = int(df_train['outliers'].mean()*df_test.shape[0])  # outlier 个数
print('outlier_num is ', outlier_num)
predictions_is_outlier['target'] =  predictions

outlier_id = list(predictions_is_outlier.sort_values(by='target',ascending = False).head(outlier_num)['card_id'])
print('len of outlier_id is ', len(outlier_id), 'first 20 of outlier_id is ', outlier_id[:20])